In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist
from keras import models
from keras.layers import Dense, Activation, Conv2D, Flatten
from keras.optimizers import Adam
import os

Using TensorFlow backend.


# Data load & preprocessing

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

x_train = x_train / 255.0
x_test = x_test / 255.0

# Build network

In [3]:
def build_net():
    model = models.Sequential()
    model.add(Conv2D(16, (3,3), padding = 'same', input_shape = (28, 28, 1)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr = 0.001)
                          ,metrics = ['accuracy'])
    return model

In [4]:
mnist_net = build_net()

# Save model after train

In [5]:
# make directory
save_dir = './save_model/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [6]:
mnist_net.fit(x_train, y_train, batch_size = 32, epochs = 2)

Epoch 1/2
60000/60000 [==============================] - 22s 362us/step - loss: 0.1939 - acc: 0.9451
Epoch 2/2
60000/60000 [==============================] - 19s 322us/step - loss: 0.0733 - acc: 0.9789


#### The save () function stores both the structure and the weight of the model.

In [7]:
mnist_net.save(save_dir + 'save_model.h5')

# saved model reuse using load_model()

import load_model

In [8]:
from keras.models import load_model

In [9]:
loaded_model = load_model(save_dir + 'save_model.h5')

show the architecture

In [10]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                125450    
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 125,610
Trainable params: 125,610
Non-trainable params: 0
_________________________________________________________________


#### The get_config () function shows detailed configuration information for the model 

In [11]:
loaded_model.get_config()

{'layers': [{'class_name': 'Conv2D',
   'config': {'activation': 'linear',
    'activity_regularizer': None,
    'batch_input_shape': (None, 28, 28, 1),
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'dtype': 'float32',
    'filters': 16,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'distribution': 'uniform',
      'mode': 'fan_avg',
      'scale': 1.0,
      'seed': None}},
    'kernel_regularizer': None,
    'kernel_size': (3, 3),
    'name': 'conv2d_1',
    'padding': 'same',
    'strides': (1, 1),
    'trainable': True,
    'use_bias': True}},
  {'class_name': 'Activation',
   'config': {'activation': 'relu',
    'name': 'activation_1',
    'trainable': True}},
  {'class_name': 'Flatten',
   'config': {'data_format': 'channels_last',
    'name': 'flatten_1',
    'trainable': True

In [12]:
loaded_model.predict(x_test)

array([[6.3333627e-09, 2.8202527e-11, 1.9573962e-07, ..., 9.9996305e-01,
        1.8114034e-06, 2.4275182e-06],
       [5.2111358e-07, 3.4793211e-06, 9.9989009e-01, ..., 5.4458408e-14,
        1.0138346e-05, 1.0442023e-12],
       [6.3531559e-05, 9.9586111e-01, 1.5023098e-03, ..., 7.1148953e-04,
        1.1111365e-03, 1.2173619e-05],
       ...,
       [4.9894902e-09, 1.4088255e-08, 2.4699592e-08, ..., 7.0802722e-05,
        5.8716381e-05, 2.6022719e-04],
       [8.6489980e-09, 3.5226779e-09, 9.8673869e-10, ..., 1.4164848e-09,
        4.7293751e-04, 8.3327931e-09],
       [2.8715714e-08, 6.3398183e-11, 2.9149945e-05, ..., 1.2070952e-09,
        6.4446418e-08, 5.1614096e-10]], dtype=float32)

In [13]:
loaded_model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 148us/step


[0.07203650470729918, 0.9776]

#### Import and reuse only weight values from saved models

In [14]:
mnist_net_only_weights = build_net()

In [15]:
mnist_net_only_weights.load_weights(save_dir + 'save_model.h5')

In [16]:
mnist_net_only_weights.predict(x_test)

array([[6.3333627e-09, 2.8202527e-11, 1.9573962e-07, ..., 9.9996305e-01,
        1.8114034e-06, 2.4275182e-06],
       [5.2111358e-07, 3.4793211e-06, 9.9989009e-01, ..., 5.4458408e-14,
        1.0138346e-05, 1.0442023e-12],
       [6.3531559e-05, 9.9586111e-01, 1.5023098e-03, ..., 7.1148953e-04,
        1.1111365e-03, 1.2173619e-05],
       ...,
       [4.9894902e-09, 1.4088255e-08, 2.4699592e-08, ..., 7.0802722e-05,
        5.8716381e-05, 2.6022719e-04],
       [8.6489980e-09, 3.5226779e-09, 9.8673869e-10, ..., 1.4164848e-09,
        4.7293751e-04, 8.3327931e-09],
       [2.8715714e-08, 6.3398183e-11, 2.9149945e-05, ..., 1.2070952e-09,
        6.4446418e-08, 5.1614096e-10]], dtype=float32)

In [17]:
mnist_net_only_weights.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 145us/step


[0.07203650470729918, 0.9776]

# only save weights

In [18]:
mnist_net_save_only_weights = build_net()
mnist_net_save_only_weights.fit(x_train, y_train, batch_size = 32, epochs = 2)

Epoch 1/2
60000/60000 [==============================] - 22s 365us/step - loss: 0.2155 - acc: 0.9383
Epoch 2/2
60000/60000 [==============================] - 23s 390us/step - loss: 0.0836 - acc: 0.9754


In [19]:
mnist_net_save_only_weights.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 148us/step


[0.07592314008828252, 0.9776]

In [20]:
mnist_net_save_only_weights.save_weights(save_dir + 'save_only_weights.h5')

In [21]:
mnist_net_save_only_weights_reuse = build_net()

In [22]:
mnist_net_save_only_weights_reuse.load_weights(save_dir + 'save_only_weights.h5')

In [23]:
mnist_net_save_only_weights_reuse.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 169us/step


[0.07592314008828252, 0.9776]

# save model as json format

#### Model information can be converted to json format and stored in memory as string data

In [24]:
mnist_json = build_net()

In [25]:
mnist_json_model = mnist_json.to_json()

In [26]:
print(mnist_json_model)

{"config": {"layers": [{"config": {"kernel_regularizer": null, "batch_input_shape": [null, 28, 28, 1], "dtype": "float32", "trainable": true, "bias_regularizer": null, "bias_constraint": null, "activity_regularizer": null, "kernel_initializer": {"config": {"seed": null, "distribution": "uniform", "mode": "fan_avg", "scale": 1.0}, "class_name": "VarianceScaling"}, "use_bias": true, "filters": 16, "kernel_constraint": null, "data_format": "channels_last", "name": "conv2d_5", "activation": "linear", "padding": "same", "strides": [1, 1], "bias_initializer": {"config": {}, "class_name": "Zeros"}, "kernel_size": [3, 3], "dilation_rate": [1, 1]}, "class_name": "Conv2D"}, {"config": {"trainable": true, "name": "activation_9", "activation": "relu"}, "class_name": "Activation"}, {"config": {"data_format": "channels_last", "name": "flatten_5", "trainable": true}, "class_name": "Flatten"}, {"config": {"trainable": true, "bias_regularizer": null, "units": 10, "activity_regularizer": null, "bias_con

#### json file write to path

In [27]:
with open(save_dir + "/mnist_json_model.json", "w") as json_file : 
    json_file.write(mnist_json_model)

#### json format model information can be reloaded and reused

### Load json and create model

In [28]:
from keras.models import model_from_json

In [29]:
# read json file from path
json_file = open(save_dir + "/mnist_json_model.json", "r")
loaded_model_json = json_file.read() 
json_file.close()

# load model from json file
saved_mnist_json_model = model_from_json(loaded_model_json)

load cnn weights

In [30]:
saved_mnist_json_model.load_weights(save_dir + 'save_only_weights.h5')

## Model information of json format should be compiled again after model load.

Makes the compile condition the same

In [31]:
# json format should be compiled
saved_mnist_json_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr = 0.001)
                          ,metrics = ['accuracy'])

In [32]:
saved_mnist_json_model.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 180us/step


[0.07592314008828252, 0.9776]

# Best model save using callback

import library

In [33]:
from keras.callbacks import ModelCheckpoint

In [34]:
model_path = save_dir + '{epoch:02d}-{val_loss:.4f}-{acc:.4f}.hd5'

# Only when val_loss is lower than before
cb_checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)

In [35]:
best_save_model = build_net()
best_save_model.fit(x_train, y_train,validation_split = 0.2, batch_size = 32, epochs = 10, callbacks = [cb_checkpoint])

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 20s 413us/step - loss: 0.2264 - acc: 0.9349 - val_loss: 0.1095 - val_acc: 0.9694

Epoch 00001: val_loss improved from inf to 0.10954, saving model to ./save_model/01-0.1095-0.9349.hd5
Epoch 2/10
48000/48000 [==============================] - 20s 415us/step - loss: 0.0867 - acc: 0.9741 - val_loss: 0.0826 - val_acc: 0.9769

Epoch 00002: val_loss improved from 0.10954 to 0.08264, saving model to ./save_model/02-0.0826-0.9741.hd5
Epoch 3/10
48000/48000 [==============================] - 20s 423us/step - loss: 0.0611 - acc: 0.9819 - val_loss: 0.0800 - val_acc: 0.9769

Epoch 00003: val_loss improved from 0.08264 to 0.07998, saving model to ./save_model/03-0.0800-0.9819.hd5
Epoch 4/10
48000/48000 [==============================] - 21s 441us/step - loss: 0.0464 - acc: 0.9859 - val_loss: 0.0720 - val_acc: 0.9800

Epoch 00004: val_loss improved from 0.07998 to 0.07199, saving model to ./sa

# Early Stopping

#### If a model with the highest performance is found in the beginning, and a model with better performance is no longer found

In [36]:
from keras.callbacks import EarlyStopping

In [37]:
# if val_loss can not increase in 3 patience(epochs), stop training
cb_early_stopping = EarlyStopping(monitor = 'val_loss', patience = 3)

# create model
earlystopping_model = build_net()
earlystopping_model.fit(x_train, y_train,validation_split = 0.2, batch_size = 32, epochs = 10, callbacks = [cb_early_stopping])

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 20s 407us/step - loss: 0.2354 - acc: 0.9325 - val_loss: 0.1139 - val_acc: 0.9684
Epoch 2/10
48000/48000 [==============================] - 23s 477us/step - loss: 0.0894 - acc: 0.9736 - val_loss: 0.0863 - val_acc: 0.9737
Epoch 3/10
48000/48000 [==============================] - 22s 449us/step - loss: 0.0620 - acc: 0.9814 - val_loss: 0.0769 - val_acc: 0.9783
Epoch 4/10
48000/48000 [==============================] - 21s 445us/step - loss: 0.0476 - acc: 0.9855 - val_loss: 0.0749 - val_acc: 0.9784
Epoch 5/10
48000/48000 [==============================] - 19s 405us/step - loss: 0.0373 - acc: 0.9888 - val_loss: 0.0794 - val_acc: 0.9781
Epoch 6/10
48000/48000 [==============================] - 19s 396us/step - loss: 0.0307 - acc: 0.9906 - val_loss: 0.0833 - val_acc: 0.9772
Epoch 7/10
48000/48000 [==============================] - 22s 450us/step - loss: 0.0238 - acc: 0.9926 - val_loss: 0.

# best model save + early stopping

In [38]:
combined_model = build_net()

In [39]:
earlystopping_model.fit(x_train, y_train,validation_split = 0.2, batch_size = 32, epochs = 10, callbacks = [cb_checkpoint,cb_early_stopping])

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 19s 403us/step - loss: 0.0191 - acc: 0.9944 - val_loss: 0.0810 - val_acc: 0.9784

Epoch 00001: val_loss did not improve from 0.07199
Epoch 2/10
48000/48000 [==============================] - 20s 420us/step - loss: 0.0159 - acc: 0.9955 - val_loss: 0.0914 - val_acc: 0.9767

Epoch 00002: val_loss did not improve from 0.07199
Epoch 3/10
48000/48000 [==============================] - 19s 390us/step - loss: 0.0128 - acc: 0.9962 - val_loss: 0.0906 - val_acc: 0.9778

Epoch 00003: val_loss did not improve from 0.07199
Epoch 4/10
48000/48000 [==============================] - 21s 435us/step - loss: 0.0101 - acc: 0.9970 - val_loss: 0.0949 - val_acc: 0.9766

Epoch 00004: val_loss did not improve from 0.07199
